In [4]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd
import numpy as np

# Store excel file locations to variables (change it as per your path to file)
file_2012=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\35. Section 8 Part 6  Health and Nutrition Food Security\2012_s6p1_f.csv"
file_2014=r"C:\Users\warra\Desktop\Freelance\data\data\FemaleMerge\1. Merging by Parts\35. Section 8 Part 6  Health and Nutrition Food Security\2014_S8P8.csv"

# Read excel files
df_2012 = pd.read_csv(file_2012)
df_2014 = pd.read_csv(file_2014, low_memory=False)


In [5]:
'''
This code block will be used to standardize column names across the years to avoid discrepancies during the merging process.
For example in the roaster data for 2013 rq21 and rq23 are not the same as rq21 and rq23 in 2014 data, but they have the same variable names. Hence, we decide to rename such columns beforehand
We will add the updated name to the mapping dictionaries instead of the original names.

'''

# Rename columns in df_2012
df_2012.rename(columns={
    'S6P1Q1A': 'FS_NEF_30d',
    'S6P1Q1B': 'FS_NEFFreq_30d',
    'S6P1Q2A': 'FS_SM_30d',
    'S6P1Q2B': 'FS_SMFreq_30d',
    'S6P1Q3A': 'FS_FM_30d',
    'S6P1Q3B': 'FS_FMFreq_30d',
    'S6P1Q4A': 'FS_NF_30d',
    'S6P1Q4B': 'FS_NFFreq_30d',
    'S6P1Q5A': 'FS_SH_30d',
    'S6P1Q5B': 'FS_SHFreq_30d',
    'S6P1Q6A': 'FS_NFDN_30d',
    'S6P1Q6B': 'FS_NFDNFreq_30d',
    'PROVINCE_ID': 'P_ID',
    'DISTRICT_ID': 'D_ID',
    'TEHSIL_ID': 'T_ID',
    'UC_ID': 'UC_ID',
    'MAUZA_ID': 'M_ID'
}, inplace=True)



# df_2014 doesn't need renaming as it is the reference

In [6]:
# Updated mappings

mapping_2012 = [
    'hid', 'round', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 
    None, None, 'FS_NEF_30d', 'FS_NEFFreq_30d', 'FS_SM_30d', 'FS_SMFreq_30d', 'FS_FM_30d', 'FS_FMFreq_30d', 
    'FS_NF_30d', 'FS_NFFreq_30d', 'FS_SH_30d', 'FS_SHFreq_30d', 'FS_NFDN_30d', 'FS_NFDNFreq_30d', 
    'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID'
]

mapping_2014 = [
    'hid', 'round', 's8p8a_q1', 's8p8a_q2', 's8p8a_q3', 's8p8a_q4', 's8p8a_q5a', 
    's8p8a_q5b', 's8p8a_q5c', 's8p8a_q5d', 's8p8a_q6', 's8p8a_q7', 's8p8a_q8', 
    's8p8b_sr', 's8p8b_food', 's8p8b_q7_ans', 's8p8c_sr', 's8p8c_food', 's8p8c_q1', 
    's8p8c_q2', None, None, None, None, None, None, None, None, None, None, None, 
    None, None, None, None, None, None
]


In [8]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col)   

for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [9]:
def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [10]:
# Usage with dataframes and mappings
dfs = [df_2012,  df_2014]
mappings = [mapping_2012,  mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0.1', 'Unnamed: 0', 'hid', 'round', 'FS_NEF_30d', 'FS_NEFFreq_30d', 'FS_SM_30d', 'FS_SMFreq_30d', 'FS_FM_30d', 'FS_FMFreq_30d', 'FS_NF_30d', 'FS_NFFreq_30d', 'FS_SH_30d', 'FS_SHFreq_30d', 'FS_NFDN_30d', 'FS_NFDNFreq_30d', 'P_ID', 'D_ID', 'T_ID', 'UC_ID', 'M_ID']
Appending data for column hid
Appending data for column round
Appending data for column FS_NEF_30d
Appending data for column FS_NEFFreq_30d
Appending data for column FS_SM_30d
Appending data for column FS_SMFreq_30d
Appending data for column FS_FM_30d
Appending data for column FS_FMFreq_30d
Appending data for column FS_NF_30d
Appending data for column FS_NFFreq_30d
Appending data for column FS_SH_30d
Appending data for column FS_SHFreq_30d
Appending data for column FS_NFDN_30d
Appending data for column FS_NFDNFreq_30d
Appending data for column P_ID
Appending data for column D_ID
Appending data for column T_ID
Appending data for column UC_ID
Appending data for column M_ID
Processing 

In [11]:
# Rename columns for the merged file (if needed)
rename_mapping = {
    'hid': 'HID',
    'round': 'Survey_Round',
    's8p8a_q1': 'FS_NEF_12m',
    's8p8a_q2': 'FS_NBM_12m',
    's8p8a_q3': 'FS_CSM_12m',
    's8p8a_q4': 'FS_CSM_Freq',
    's8p8a_q5a': 'FS_CSMAF_12m',
    's8p8a_q5b': 'FS_CSMAM_12m',
    's8p8a_q5c': 'FS_CSMFC_12m',
    's8p8a_q5d': 'FS_CSMMC_12m',
    's8p8a_q6': 'FS_HHVH_12m',
    's8p8a_q7': 'FS_HHBF_12m',
    's8p8a_q8': 'FS_HHBFNoT_12m',
    's8p8b_sr': 'FS_FoodCode',
    's8p8b_food': 'FS_FoodName',
    's8p8b_q7_ans': 'FS_Ans',
    's8p8c_sr': 'FS_ItemCode',
    's8p8c_food': 'FS_ItemName',
    's8p8c_q1': 'FS_ItemENoD_7d',
    's8p8c_q2': 'FS_ItemMSrc_7d'
}

merged_df.rename(columns=rename_mapping, inplace=True)
'''
# Drop redundant columns
merged_df.drop(merged_df.columns[merged_df.columns.str.contains('Unnamed', case=True)], axis=1, inplace=True)
merged_df.drop(merged_df.columns[merged_df.columns.str.contains(' ', case=False)], axis=1, inplace=True)
'''
# Save the merged dataframe to a CSV file
merged_df.to_csv('8. merged_HnN_Food Security.csv', index=False)
